# Healthy Streets of Los Angeles Mobility plan analysis

This project finds out how well LA Mobility Plan 2035 was implemented to this day (November 2023) for protected and unprotected bike lanes and protected bus lanes. Also it finds the same data for Neighborhood Enhanced Network bike lanes.

Sources:
* Current Bike Lanes: https://geohub.lacity.org/datasets/ladot::city-of-los-angeles-bikeways-1/explore
* Mobility Plan bike lanes: https://geohub.lacity.org/datasets/bicycle-enhanced-network/explore 
* Mobility Plan bike paths: https://geohub.lacity.org/datasets/lahub::green-network-bicycle-paths-1/explore </br >
Mobility Plan bike lanes tiers are in "Bicycle_N" column
* Neighborhood Enhanced Network bike lanes https://geohub.lacity.org/datasets/lahub::neighborhood-enhanced-network/explore
* Current Bus Lanes: https://www.google.com/maps/d/u/1/edit?mid=10co-9X_jGrJtGBA9qVhwWOqUpQNng2dx&usp=sharing 
* Mobility plan Bus lanes: https://geohub.lacity.org/datasets/transit-enhanced-network/explore?location=34.018131%2C-118.376481%2C11.68

Compare: 
* Actual Bus Lanes = Mobility Plan tier 3
* Actual Bike Lanes Class 4 = Protected (Mobility Plan tier 1)
* Actual Bike Lanes Class 2 = Unprotected (Mobility Plan tier 2 and 3)
* Actual Bike Lanes Class 1 = Bicycle Enhanced Network
* Actual Bike Lanes Class 3 = Neighborhood Enhanced Network

Deliverable: 
* What % of the mobility plan has been implemented
* Geojson result
* year > 2015, because Mobility Plan was approved in late 2015

Any questions - @sunchugasheva

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import folium
import datetime

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)

In [ ]:
today = datetime.datetime.now().strftime("%Y_%m_%d")
print(today)

## functions

In [ ]:
def show_map(gdf1, name1, color1, gdf2, name2, color2):
    '''print a movable map for two geodataframes'''
    print(
            f'{color1}: {name1}, {color2}: {name2}'
        )
    
    f = folium.Figure(width=1000, height=500)
    
    m = gdf1.explore(
        name = name1,
        color = color1
    ).add_to(f)

    map_2 = gdf2.explore(
        m=m,  # pass the map object
        name = name2,
        color = color2
    )

    folium.TileLayer(
        'CartoDB positron',
        show=False
    ).add_to(m) 

    folium.LayerControl().add_to(m)
    
    return m

In [ ]:
def get_xy(line):
    '''if linestring has x, y, z coords get x, y only'''
    if line.has_z:
        xy_line = [xy[:2] for xy in list(line.coords)]
    return LineString(xy_line)

In [ ]:
def buffer(gdf, radius, proj='EPSG:4326', proj_calc='EPSG:3857'):
    '''
    convert a gdf of linestrings into a gdf of polygons with radius
    - gdf - GeoDataFrame, has column "geometry"
    - radius - radius of bufferm meters
    - proj - projection of the original dataset
    - proj_calc='EPSG:3857' - projection for calculations
    '''  
    gdf = gdf.copy()
    gdf_calc = gdf.to_crs(proj_calc)
    #print('data proj:', proj, '\ncalculation proj: ', proj_calc)
    gdf['buffered'] = gdf_calc.buffer(radius, cap_style=2).to_crs(proj)
    gdf.set_geometry('buffered', inplace=True)
    
    # merge all intersecting buffered polygons
    gdf_return = gpd.GeoDataFrame(
        geometry=[gdf.unary_union]
    ).explode(
        index_parts=False
    ).reset_index(
        drop=True
    )
    gdf_return.geometry.crs = proj
    
    return gdf_return

In [ ]:
def get_lengths(gdf, column='geometry', proj='EPSG:4326', proj_calc='EPSG:3857'):
    '''
    return a gdf with a column of linestring length in proj_calc units (m)
    - gdf - GeoDataFrame
    - column - column to be used for length calculation if not "geometry"
    - proj - projection of the original dataset
    - proj_calc='EPSG:3857' - projection for calculations
    '''
    if column:
        gdf.set_geometry(column, inplace=True)
    # convert projection to proj_calc, if default - the units will be in meters
    gdf_m = gdf.to_crs(proj_calc)
    gdf['length_m'] = gdf_m.length
    
    return gdf

In [ ]:
def compare_length(
        name,
        gdf_plan,
        gdf_actual,
        conditions,
        radius = 10,
        year = None,
        column_year = 'Year_',
        print_map = True,
        test_map = False
    ):
    '''
    find implemented parts of mobility plan (overlapping buffered actual geodata
    and planned geodata), then calculate the percentage ofimplemented/planned 
    lengths of paths, return both geodf and percentage:
    - name 
    - gdf_plan - GeoDataFrame of Mobility plan
    - gdf_actual - GeoDataFrame of existing roads, has a column_year
    - conditions - filter specific types of bike/bus lanes
    - radius - radius in meters to widen the line of actual roads
    - year - filter data after construction year
    - column_year - column of construction year
    - print_map - show final map
    - test_map - show interim map
    '''
    print(f'{name} for year after {year}:')
    gdf_plan = gdf_plan.copy()
    gdf_actual = gdf_actual.copy()
    
    if year:
        gdf_actual = gdf_actual[gdf_actual[column_year]>year].copy()
    if conditions['plan']:
        if 'bike' in key:
            gdf_plan = gdf_plan[gdf_plan.BICYCLE_N.isin(
                conditions['plan']
            )].copy()
        if 'bus' in key:
            gdf_plan = gdf_plan[gdf_plan.TRANSIT_N.isin(
                conditions['plan']
            )].copy()
    if conditions['actual']:
            gdf_actual = gdf_actual[gdf_actual.Class.isin(
                conditions['actual']
            )].copy()
    
    print(f'radius = {radius} m')
    # widen linestring to polygon
    gdf_actual_buffer = buffer(
        gdf_actual, 
        radius = radius,
        proj = gdf_actual.geometry.crs
    )
    if test_map:
        display(
            show_map(
                gdf2 = gdf_actual_buffer[['geometry']],
                name2 = 'implemented', 
                color2 = 'blue',
                gdf1 = gdf_plan[['OBJECTID', 'geometry']],
                name1 = 'plan', 
                color1 = 'green',
            )
        )
    
    # intersect polygons of actual and planned paths
    gdf_implemented = gpd.overlay(
        gdf_plan, 
        gdf_actual_buffer, 
        how='intersection',
        keep_geom_type=False
    )
    gdf_implemented = get_lengths(gdf_implemented)
    gdf_implemented = gdf_implemented[
        round(gdf_implemented.length_m/(radius*2), 2)>1.01
    ].reset_index(drop=True)
    
    # map of buffered planned (green) and implemented (red) paths
    if print_map:
        display(
            show_map(
                gdf2 = gdf_implemented[['OBJECTID', 'geometry']],
                name2 = 'implemented', 
                color2 = 'red',
                gdf1 = gdf_plan[['OBJECTID', 'geometry']],
                name1 = 'plan', 
                color1 = 'green',
            )
        )
        
    print(
        'since', year, 'year:',
        round(get_lengths(gdf_implemented).length_m.sum()/1609.34, 2), 'miles implemented of ',
        round(get_lengths(gdf_plan).length_m.sum()/1609.34, 2), 'miles planned',
        # '\nimplemented lanes records:', gdf_implemented.shape[0],
        # '\nplanned lanes records:', gdf_plan.shape[0]        
    )
    
    percentage = round(
        get_lengths(gdf_implemented).length_m.sum()/
        get_lengths(gdf_plan).length_m.sum()*100, 
        2
    )

    return percentage, gdf_implemented

In [ ]:
def get_unimplemented(gdf_plan, gdf_implemented, radius=15, print_map=False):
    '''
    find unimplemented parts of mobility plan (overlapping buffered implemented
    geodata and planned geodata):
    - gdf_plan - GeoDataFrame of Mobility plan
    - gdf_implemented - GeoDataFrame of implemented roads
    - radius - radius in meters for the buffer
    - print_map - show final map
    '''
    implemented_buffer = buffer(gdf_implemented, radius = radius)
    gdf_unimplemented = gpd.overlay(
        gdf_plan, 
        implemented_buffer, 
        how='difference',
        keep_geom_type=False
    )
    unimpl_clean = get_lengths(gdf_unimplemented)
    unimpl_clean = unimpl_clean[unimpl_clean.length_m>2*radius].copy()
    if print_map:
        m = show_map(
            gdf1 = gdf_plan, 
            name1 = 'plan', 
            color1 = 'blue', 
            gdf2 = unimpl_clean, 
            name2 = 'un', 
            color2 = 'red'
        )
        display(m)
       
    return unimpl_clean

## bike lanes

In [ ]:
bike_plan_file = open('Bicycle_Enhanced_Network.geojson')
bike_plan_geo_raw = gpd.read_file(bike_plan_file)
bike_actual_file = open('City_of_Los_Angeles_Bikeways.geojson')
bike_actual_geo_raw = gpd.read_file(bike_actual_file)

In [ ]:
bike_plan_geo = bike_plan_geo_raw.copy()
bike_actual_geo = bike_actual_geo_raw.copy()
print(
    'Mobility plan records:',
    bike_plan_geo.shape[0],
    '\nActual bike lanes records:',
    bike_actual_geo.shape[0],
    '\nActual lanes without construction year:',
    bike_actual_geo[bike_actual_geo.Year_.isnull()].shape[0]
)
display(bike_plan_geo.head(1))
display(bike_actual_geo.tail(1))

### Clean up data/figure out intersections

In [ ]:
# actual bike paths surprisingly have z coordinate,
# so following is the best way to get rid of it (all z==0) I came up with

bike_actual_geo = bike_actual_geo.explode(ignore_index=False, index_parts=False)
bike_actual_geo.geometry = bike_actual_geo.geometry.apply(lambda x: get_xy(x))
display(bike_actual_geo.head(1))

In [ ]:
# save processed actual data for future reference
bike_actual_geo.to_file('bike_actual.geojson', driver='GeoJSON')

<b>Geo intersecting data</b></br>
* two linestrings</br>
As we see below, lines for planned and actual bike paths are not exactly the same (but actually close, see the map below).
* a linesting and polygon</br>
We can artificially widen one of the linestrings into a polygon so it would overlap with the other linestring - see function 'buffer'.

We chose converting to polygon actual data over planned for following reasons:
1. we won't double count bike paths, planned data has one line where actual data sometines two (one bike lane one way and one lane the other way).
2. we're comparing result (implemented bike lanes) to planned data, so there we won't depend on possible irregularities of existing bike paths</br>

In [ ]:
# show_map(
#     gdf1 = bike_actual_geo[['SECT_ID', 'geometry']],
#     name1 = 'actual', 
#     color1 = 'blue',
#     gdf2 = bike_plan_geo[['OBJECTID', 'geometry']],
#     name2 = 'plan', 
#     color2 = 'green',
# )

### Calculate percentages for Class 4, 2: protected/unprotected lanes

In [ ]:
bike_conditions_class24 = {
    'protected bike lane': {'plan': [1], 'actual': [4]},
    'unprotected bike lane': {'plan': [2, 3], 'actual': [2]}
}
results = []
result_gdfs = []

for key in bike_conditions_class24.keys():
    for year in (None, 2015):
        percentage, result_gdf = compare_length(
            name = key,
            conditions = bike_conditions_class24[key],
            gdf_actual = bike_actual_geo[[
                            'OBJECTID_12',
                            'SECT_ID',
                            'Class',
                            'Year_',
                            'geometry'
                        ]],
            gdf_plan = bike_plan_geo[[
                            'OBJECTID',
                            'BICYCLE_N',
                            'geometry'
                       ]],
            print_map = False,
            test_map = False,
            year = year,
            column_year = 'Year_'
        )
        results.append([key, year, percentage])
        result_gdfs.append([key, year, result_gdf])
        
        print(
            percentage,
            f'% of {key} implemented after {year}',
            '\n'
        )
        
    print('\n----')    

### Bike Paths vs class 1

In [ ]:
bike_plan_paths = open('Bicycle_Enhanced_Network_Paths.geojson')
bike_paths_geo_raw = gpd.read_file(bike_plan_paths)

In [ ]:
bike_path_geo = bike_paths_geo_raw.copy()
print(bike_path_geo.shape[0])
display(bike_path_geo.head())

Geometry for the city's bike path file is too inaccurate to run a spatial comparison i.e. it misses the majority of Ballona Creek </br>
Fortunately there are no bike paths completed that aren't on the mobility plan, so we can use the city's bikeway map of Class 1 routes and sort by year and just compare total lengths</br>
<span style="color:red">(this done in the next couple of cells, but possible solution is to widen the buffer area - parameter radius, see below)</span>

In [ ]:
# # return length of all planned path
# planned_lengths = get_lengths(bike_path_geo)
# m_planned = planned_lengths.length_m.sum()
# 
# # return length of all class 1 routes
# actual_paths = bike_actual_geo.loc[bike_actual_geo['Class'] == 1].copy()
# path_built = get_lengths(actual_paths)
# m_built = path_built.length_m.sum()
# 
# # return length of all class 1 routes completed after 2015
# actual_paths_2015 = actual_paths.loc[actual_paths['Year_'] > 2015].copy()
# path_built_2015 = get_lengths(actual_paths_2015)
# m_built_2015 = path_built_2015.length_m.sum()
# 
# print(round(m_built/m_planned * 100, 2), '% of bike paths implemented total ')
# print(round(m_built_2015/m_planned * 100, 2), '% of bike paths implemented after 2015 ')

^ output </br>
37.37 % of bike paths implemented total </br>
2.74 % of bike paths implemented after 2015 </br>

In [ ]:
# bike_path_conditions = {
#     'bike path': {'plan': None, 'actual': [1]},
# }

# for key in bike_path_conditions.keys():
#     for year in (None, 2015):
#         percentage, result_gdf = compare_length(
#             name = key,
#             conditions = bike_path_conditions[key],
#             gdf_actual = bike_actual_geo[[
#                             'OBJECTID_12',
#                             'SECT_ID',
#                             'Class',
#                             'Year_',
#                             'geometry'
#                         ]],
#             gdf_plan = bike_path_geo,
#             print_map = True,
#             year = year,
#             column_year = 'Year_',
#             test_map = False  
#         )
        
#         results.append([key, year, percentage])
#         result_gdfs.append([key, year, result_gdf])

#         print({key})
        
#         print(
#             percentage,
#             f'% of {key} implemented after {year}',
#             '\n'
#         )
        
#     print('\n----')    

^ output:</br>
bike path</br>
since None year: </br>
implemented lanes records: 45 </br>
planned lanes records: 146</br>
{'bike path'}</br>
11.55 % of bike path implemented after None </br>
</br>
bike path</br>
since 2015 year: </br>
implemented lanes records: 3 </br>
planned lanes records: 146</br>
{'bike path'}</br>
0.61 % of bike path implemented after 2015 </br>

In [ ]:
bike_conditions_class1 = {
    'class1 bike lane': {'plan': None, 'actual': [1]},
}

for key in bike_conditions_class1.keys():
    for year in (None, 2015):
        percentage, result_gdf = compare_length(
            name = key,
            gdf_plan = bike_path_geo[['OBJECTID', 'geometry']],
            gdf_actual = bike_actual_geo[[
                'OBJECTID_12', 'Year_', 'Class','geometry'
            ]],
            conditions = bike_conditions_class1[key],
            radius = 120,
            print_map = True,
            test_map = True,
            year = year,
            column_year = 'Year_'
                )
        print(f'{percentage}% of bike path implemented after {year}')
        results.append([key, year, percentage])
        result_gdfs.append([key, year, result_gdf])
        print('_______')

### NEN vs class 3

In [ ]:
bike_nen_file = open('Neighborhood_Enhanced_Network.geojson')
bike_nen_geo_raw = gpd.read_file(bike_nen_file)

In [ ]:
bike_nen_geo = bike_nen_geo_raw.copy()
print(bike_nen_geo.shape[0])
display(bike_nen_geo.head())

In [ ]:
bike_conditions_class3 = {
    'NEN bike lane': {'plan': None, 'actual': [3]},
}

for key in bike_conditions_class3.keys():
    for year in (None, 2015):
        percentage, result_gdf = compare_length(
            name = key,
            conditions = bike_conditions_class3[key],
            gdf_actual = bike_actual_geo[[
                            'OBJECTID_12',
                            'SECT_ID',
                            'Class',
                            'Year_',
                            'geometry'
                        ]],
            gdf_plan = bike_nen_geo,
            print_map = False,
            year = year,
            column_year = 'Year_',
            test_map = False
        )
        results.append([key, year, percentage])
        result_gdfs.append([key, year, result_gdf])
        
        print(
            percentage,
            f'% of {key} implemented after {year}',
            '\n'
        )
        
    print('\n----')    

## bus lanes

In [ ]:
bus_plan_file = open('Transit_Enhanced_Network.geojson')
bus_plan_geo_raw = gpd.read_file(bus_plan_file)
bus_actual_file = open('bus_only_lanes_sat.geojson')
bus_actual_geo_raw = gpd.read_file(bus_actual_file)

In [ ]:
bus_plan_geo = bus_plan_geo_raw.copy()
bus_actual_geo = bus_actual_geo_raw.copy()
print(
    'Mobility plan records:',
    bus_plan_geo.shape[0],
    '\nActual bus lanes records:',
    bus_actual_geo.shape[0]
)
display(bus_plan_geo.head())
display(bus_actual_geo.head())

In [ ]:
# take a look at planned vs existing protected lanes
protected_bus_plan = bus_plan_geo[bus_plan_geo.TRANSIT_N >= 2].copy()
display(
            show_map(
                gdf2 = bus_actual_geo[['Name', 'geometry']],
                name2 = 'actual', 
                color2 = 'blue',
                gdf1 = protected_bus_plan[['OBJECTID', 'geometry']],
                name1 = 'plan', 
                color1 = 'green',
            )
        )

In [ ]:
bus_conditions = {
    'bus lane': {'plan': [3, 2], 'actual': None},
}

for key in bus_conditions.keys():
    for year in (None, 2015):
        percentage, result_gdf = compare_length(
            name = key,
            conditions = bus_conditions[key],
            radius = 20,
            gdf_actual = bus_actual_geo[[
                            'Name',
                            'geometry',
                            'Year_',
                            'Hours'
                        ]].copy(),
            gdf_plan = bus_plan_geo,
            print_map = False,
            test_map = False,
            year = year,
            column_year ='Year_' 
        )
        results.append([key, year, percentage])
        result_gdfs.append([key, year, result_gdf])
    
        print(
            percentage,
            f'% of {key} implemented after {year}',
            '\n'
        )
        
        print('\n----')    

## save results

### implemented

In [ ]:
# results_df = pd.DataFrame(
#     columns=['transport type', 'year', 'implemented_perc'],
#     data = results
# )
# display(results_df)

In [ ]:
# results_df.to_csv(f'mobility_plan_percentage_implemented_{today}.csv', index=False)

In [ ]:
# for df in result_gdfs:
#     name = df[0].replace(' ', '_') + '_implemented'

#     year = df[1]
#     if year:
#         name = name + f'_since_{year}'
#     print(name)
#     df[2].to_file(f'{name}_{today}.geojson', driver='GeoJSON')

### unimplemented

* Actual Bike Lanes Class 4 = Protected (Mobility Plan tier 1)
* Actual Bike Lanes Class 2 = Unprotected (Mobility Plan tier 2 and 3)
* Actual Bike Lanes Class 1 = Bicycle Enhanced Network
* Actual Bike Lanes Class 3 = Neighborhood Enhanced Network

In [ ]:
gdfs_list = []
for df in result_gdfs:
    gdfs_list.append(df[0])

bike_conditions = {**bike_conditions_class24,
                   **bike_conditions_class1,
                   **bike_conditions_class3,
                  **{'bus lane': None}}

In [ ]:
unimplemented_gdfs = []
for key in bike_conditions.keys():
    for df in result_gdfs:
        if df[0]==key:
            name = df[0].replace(' ', '_') + '_unimplemented'
            year = df[1]
            if year:
                name = name + f'_since_{year}'
            print(name)
            gdf_impl = df[2]

            if key in ['protected bike lane', 'unprotected bike lane']:
                gdf_plan = bike_plan_geo[bike_plan_geo.BICYCLE_N.isin(
                                bike_conditions_class24[key]['plan']
                            )].copy()
            if key=='class1 bike lane': 
                gdf_plan = bike_path_geo.copy()
            if key=='NEN bike lane':
                gdf_plan = bike_nen_geo.copy()
            if key=='bus lane':
                gdf_plan = protected_bus_plan.copy()
            
            gdf_unimpl = get_unimplemented(
                gdf_plan = gdf_plan[['OBJECTID', 'geometry']],
                gdf_implemented = gdf_impl,
                radius = 20,
                print_map = False
            )
            unimplemented_gdfs.append([name, gdf_unimpl])

In [ ]:
# # save geodataframes
# for df in unimplemented_gdfs:
#     print(df[0])
#     display(df[1].shape)
#     df[1].to_file(f'{df[0]}_{today}.geojson', driver='GeoJSON')